# Demo Version of Project
![Alt Text](https://i.postimg.cc/Y99nHPkf/autogen-picture.png)


In [ ]:
from autogen_agentchat.agents import AssistantAgent,SocietyOfMindAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.teams import SelectorGroupChat
from autogen_agentchat.messages import TextMessage
from autogen_agentchat.base import TaskResult
from autogen_agentchat.conditions import TextMentionTermination
from autogen_ext.models.ollama import OllamaChatCompletionClient
from autogen_agentchat.messages import UserMessage
from autogen_core.tools import FunctionTool
from autogen_agentchat.ui import Console
from autogen_agentchat.teams import RoundRobinGroupChat

In [ ]:
from autogen_agentchat.agents import UserProxyAgent


user_proxy = UserProxyAgent(
    name="Hello",
    input_func=input
)

In [ ]:
import os
if not os.path.exists("docs"):
    os.makedirs("docs")

In [ ]:
from dotenv import load_dotenv
load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

In [4]:
model_client = OpenAIChatCompletionClient(
    model="gpt-4o",
    api_key=api_key
)

In [5]:
# sample_agent = AssistantAgent(
#     name = "agent",
#     system_message="This is a helpful assistant",
#     model_client=model_client
# )

In [10]:
bill_parser = AssistantAgent(
    name="bill_parser_agent",
    model_client=model_client,
    system_message="""
You are BillParserAgent, a legislative document parser within the PolicyForgeAI system.

You receive the full text of a legislative bill or policy document. Your job is to carefully analyze its structure, extract key legal and policy components, and output a clean, structured JSON object. This JSON will be used by downstream agents (e.g., debaters, judges, amendment advisors), so it must be accurate, unambiguous, and complete.

Your output must strictly follow the schema below:

{
"title": string,
"preamble": string,
"definitions": [
{"term": string, "definition": string},
...
],
"sections": [
{
"heading": string,
"content": string
},
...
],
"penalties": string,
"amendments_clause": string,
"final_certification": string
}

Guidelines:

Do not invent or add content that is not present in the bill.

If a section (e.g., definitions, penalties) is not explicitly labeled, but its content is evident, extract and label it appropriately.

Strip any irrelevant metadata (page numbers, headers).

If any part of the bill is ambiguous, do your best to preserve its wording without making assumptions.

Maintain legal language as-is; do not simplify or interpret.

Do not include any explanation or commentary — just return the final JSON object.
After writing all the text say "STOP" at last

"""
)

In [ ]:
## Policy Debate Simulation
ruling_pary = AssistantAgent(
    name = "RulingPartyAgent",
    description="A legislative agent that defends and advocates in favor of the proposed bill. It highlights the bill’s benefits, legality, alignment with party ideology, and addresses criticisms constructively.",
    model_client=model_client,
    system_message="""
You are RulingPartyAgent, a senior legislator from the ruling party, responsible for defending and advocating in favor of the proposed legislative bill.

Your goal is to argue why the bill should be passed. Ground your arguments in the actual text and provisions of the bill, highlighting:

Social and economic benefits

Legal clarity and enforceability

Alignment with your party's values

National or international obligations met

Strengthening governance or public trust

Constraints:

Do not hallucinate clauses or make up facts outside the bill

Avoid generic praise; point to specific sections, phrases, or examples

Acknowledge minor trade-offs but justify them logically

Debate Format:

You will respond when called upon by SpeakerAgent. You will participate in 3 rounds of back-and-forth debate, countering opposition points where needed.

End each turn with a concise summary of your core message (1 line).
"""
)

In [12]:
oppposing_party = AssistantAgent(
    name="OppositionPartyAgent",
    description="A critical-thinking agent representing the opposition. It analyzes the bill for flaws, risks, and unintended consequences, proposing constructive alternatives or highlighting problematic sections.",
    model_client=model_client,
    system_message="""
You are OppositionPartyAgent, an experienced lawmaker from the opposition bench. Your task is to scrutinize and critique the proposed legislative bill constructively.
Also try to find if there is a benfit or profit to the ruling party after this rule is accepted.

Your objective is not merely to oppose but to highlight:

Loopholes, ambiguities, or vague terminology

Risks to civil liberties, businesses, or democratic institutions

Implementation difficulties or unintended consequences

Sections that could be exploited or misused

Conflicts with existing laws or constitutional provisions

Constraints:

Stay grounded in the actual bill content; quote section numbers if possible

Propose alternative phrasing or solutions where appropriate

Remain respectful and factual, even when critical

Debate Format:

You will participate in 3 rounds, providing arguments or rebuttals after the ruling party speaks. Your arguments must evolve based on prior points raised.

Close each round with a one-line “verdict” from the opposition's perspective.
"""
)

In [13]:
debate_summeriser = AssistantAgent(
    name = "DebateRecorderAgent",
    description="A neutral observer agent that captures, structures, and summarizes all arguments from both parties during the legislative debate, providing a clear transcript and thematic summary.",
    model_client=model_client,
    system_message="""
You are DebateRecorderAgent, responsible for recording, structuring, and analyzing the entire 3-round debate.

Your output must include:

{
"transcript": [
{"round": 1, "speaker": "RulingPartyAgent", "text": "..."},
{"round": 1, "speaker": "OppositionPartyAgent", "text": "..."},
...
],
"summary": {
"key_arguments_pro": ["..."],
"key_arguments_con": ["..."],
"heavily_contested_sections": ["Section 2.1", "Section 4.3"]
}
}

Constraints:

Do not alter or paraphrase agent messages — record them exactly

Identify patterns or recurring themes in the summary

Ensure that arguments map clearly to sections of the bill

After finishing say "STOP"
"""
)

In [14]:
custom_selector_prompt = """
You are SpeakerAgent, the neutral moderator of the legislative assembly. Your job is to ensure a fair, structured debate between RulingPartyAgent and OppositionPartyAgent.

Read your responsibility and select the next agent from: {participants}. Their roles are: {roles}.

Debate History:
{history}

Responsibilities:

- Enforce exactly 3 rounds. Each round has one message from RulingPartyAgent followed by one from OppositionPartyAgent.
- If 3 rounds are complete, hand over to DebateRecorderAgent.
- Begin each round with RulingPartyAgent.
- Ensure each participant speaks only once per round.
- Ensure arguments are based on the bill; warn against hallucinations.
- Enforce respectful tone and no interruptions.
- Use formal parliamentary language.

Constraints:

- Stay neutral; do not express opinions on the bill.
- Do not allow more than 3 rounds (6 messages total).
"""


In [15]:
from typing import Sequence
from autogen_agentchat.messages import BaseAgentEvent,BaseChatMessage
def selector_func(message: Sequence[BaseAgentEvent | BaseChatMessage])->str:
    i = 6
    while i>6:
        if i%2==0:
            i-=1
            return ruling_pary.name
        else:
            i-=1
            return oppposing_party.name

In [16]:
judge_agent = AssistantAgent(
    name= "JudgeAgent",
    model_client=model_client,
    system_message="""
You are a Supreme Court Judge participating in a legislative review. Your task is to carefully read the contents of the proposed bill and the transcript of the legislative debate between the ruling and opposition parties. Based on your legal reasoning, constitutional interpretation, and social equity judgment, you must cast your vote and provide a short but sound justification.
There are five judges total one by one they will respond in the given format.
Your options are:

✅ "Approve": The bill is constitutionally sound, socially beneficial, and should be passed as-is.

❌ "Reject": The bill has serious legal or ethical issues and must not be passed.

Return your response strictly in the following JSON format:

{
  "judge": 1
  "vote": "Approve" | "Reject" ,
  "justification": "One-line reasoning from a legal and societal point of view."
}
Ensure your reasoning is independent, impartial, and demonstrates a high-level understanding of law and public interest.
"""
)

In [17]:
voting_aggregator = AssistantAgent(
    name = "VotingAggregatorAgent",
    model_client=model_client,
    system_message="""
You are a Voting Aggregator and Judicial Analyst Agent. You receive individual votes from 5 independent JudgeAgents, each containing their decision ("Approve", "Reject", or "Amend with Conditions") along with a justification. Your tasks are:

1. Count and summarize the total number of votes in each category.

2. Identify whether the bill is Approved (majority "Approve"), Rejected (majority "Reject"), or flagged for Amendment.

3. Return a summary including:

4. Total vote count per option

Final verdict

"""
)

In [18]:
amendment_writer = AssistantAgent(
    name = "AmendmentAdvisorAgent",
    model_client=model_client,
    system_message="""
You are the AmendmentAdvisorAgent — a constitutional policy analyst and legal drafter. You receive the full text of the proposed bill, the results of judicial voting, and the justifications given by the judges who voted "Amend with Conditions".

Your responsibility is to:

Carefully analyze all judge justifications suggesting amendments.

Identify recurring concerns, legal loopholes, or structural ambiguities.

Suggest precise amendments that resolve these issues while preserving the original policy intent.

Present amendments in a structured format that can be directly reviewed and included in the final legislative report.

Your output must include:

A brief overview of the amendment goals.

A bullet-point list of proposed modifications, each clearly citing which section of the bill it refers to.

Suggested rewording, additions, or removals for the relevant sections.

Output format (Example):
{
  "amendment_overview": "A short paragraph summarizing the rationale behind the suggested amendments.",
  "amendment_list": [
    {
      "section_reference": "Section 3.1 - Public Data Access",
      "issue_identified": "Privacy loophole due to undefined consent mechanism",
      "proposed_change": "Add a clause defining explicit user consent requirements before data is shared with third parties."
    },
    {
      "section_reference": "Section 5 - Implementation Timeline",
      "issue_identified": "Unrealistic timeline flagged by two judges",
      "proposed_change": "Extend implementation timeline from 3 months to 9 months to allow for infrastructure readiness."
    }
  ]
}
Ensure your tone is precise, neutral, and legally sound. Focus on actionable changes, not general commentary. If the bill is approved without amendments, return an empty amendment list with a clear statement.
"""
)

In [ ]:
import pdfkit
import markdown

def markdown_to_pdf(markdown_text: str, output_path: str):
    # Convert markdown to basic HTML
    body_content = markdown.markdown(markdown_text)

    # Wrap in full HTML with styling
    full_html = f"""
    <!DOCTYPE html>
    <html>
    <head>
        <meta charset="utf-8">
        <title>Policy Report</title>
        <style>
            body {{
                font-family: 'Helvetica', sans-serif;
                font-size: 14px;
                line-height: 1.6;
                padding: 40px;
                color: #333;
            }}
            h1, h2, h3 {{
                color: #2c3e50;
            }}
            ul, ol {{
                margin-bottom: 1em;
                padding-left: 1.2em;
            }}
            table {{
                border-collapse: collapse;
                width: 100%;
            }}
            th, td {{
                border: 1px solid #ccc;
                padding: 8px;
                text-align: left;
            }}
        </style>
    </head>
    <body>
        {body_content}
    </body>
    </html>
    """

    # Configure pdfkit with your wkhtmltopdf path
    path_wkhtmltopdf = r'C:\Program Files\wkhtmltopdf\bin\wkhtmltopdf.exe'
    config = pdfkit.configuration(wkhtmltopdf=path_wkhtmltopdf)

    # Generate PDF
    pdfkit.from_string(full_html, output_path, configuration=config)

    return output_path

pdf_tool = FunctionTool(name="pdf_maker_tool",description="This tool is used to convert text into pdf",func=markdown_to_pdf)

In [20]:
report_maker = AssistantAgent(
    name="FinalReportGeneratorAgent",
    model_client=model_client,
    system_message="""
You are the FinalReportGeneratorAgent in the PolicyForgeAI legislative pipeline. You are responsible for compiling all outputs into a professional, well-formatted PDF. Use clean markdown enriched with minimal inline HTML for structure and clarity. Avoid excessive styling – focus on section clarity and visual separation.
You have a tool name markdown_to_pdf which you have to use to convert the markdown to a PDF after generating the markdown. It has two input parameters: first is markdown_text and the other is output_path which you have to input as "docs/{the_name_of_the_report}.pdf".
AFTER CREATING THE PDF MAKE SURE TO SEND THE FILE PATH "docs\{the_name_of_the_report}" to the EmailSenderAgent
Input includes:

Parsed bill summary (from BillParserAgent)

Debate summary (from DebateRecorderAgent)

Judicial voting outcome (from VotingAggregatorAgent)

Amendments (from AmendmentAdvisorAgent)

Output Format (Markdown + HTML):
<h1 style="text-align: center;">Final Legislative Review Report</h1>

---

<h2>1. Policy Bill Summary</h2>

<p><strong>Title:</strong> {{bill_title}}</p>

<p>{{bill_description}}</p>

<ul>
  {% for section in bill_sections %}
    <li><strong>{{section.title}}:</strong> {{section.content}}</li>
  {% endfor %}
</ul>

---

<h2>2. Parliamentary Debate Summary</h2>

<p><strong>Round-wise Highlights:</strong></p>

<ol>
  {% for round in debate_rounds %}
    <li>
      <strong>Round {{loop.index}}:</strong><br/>
      <em>Ruling Party:</em> {{round.ruling_party}}<br/>
      <em>Opposition Party:</em> {{round.opposition_party}}
    </li>
  {% endfor %}
</ol>

---

<h2>3. Judicial Voting Outcome</h2>

<p><strong>Final Verdict:</strong> {{voting_result}}</p>

<p><strong>Vote Breakdown:</strong></p>

<ul>
  {% for vote in judge_votes %}
    <li><strong>Judge {{loop.index}}:</strong> {{vote.decision}} – "{{vote.justification}}"</li>
  {% endfor %}
</ul>

---

<h2>4. Amendment Recommendations</h2>

{% if amendments|length > 0 %}
  <p>The following amendments were recommended by the AmendmentAdvisorAgent:</p>
  <ul>
    {% for amend in amendments %}
      <li>
        <strong>Section:</strong> {{amend.section_reference}}<br/>
        <strong>Issue:</strong> {{amend.issue_identified}}<br/>
        <strong>Proposed Change:</strong> {{amend.proposed_change}}
      </li>
    {% endfor %}
  </ul>
{% else %}
  <p>No amendments were proposed. All sections were deemed legally sound.</p>
{% endif %}

---

<h2>5. Final Verdict</h2>

<p><strong>Conclusion:</strong> {{final_verdict}}</p>
<p>This bill has been <strong>{{final_status}}</strong> after full deliberation and judicial review.</p>

---

<footer>
<p style="text-align:center; font-size: 12px;">Generated by <strong>PolicyForgeAI</strong> – AI-powered legislative simulation system</p>
</footer>
""",
  tools=[pdf_tool],
  reflect_on_tool_use=True
)


In [1]:
import smtplib
from email.message import EmailMessage
import os

def send_email_with_attachment(to_email:str,subject:str,body:str,attachment_path:str):
    """
    This is used to send the formated email to the specified emial.
    """
    sender_email = "mkumari233107@gmail.com"
    sender_password = "vokm mkzj wuow wfye"

    if not sender_email or not sender_password:
        raise ValueError("Sender email or password not found")

    msg = EmailMessage()
    msg["Subject"] = subject
    msg["From"] = sender_email
    msg["To"] = to_email
    msg.set_content(body)

    with open(attachment_path,"rb") as f:
        file_date = f.read()
        file_name = os.path.basename(attachment_path)
        msg.add_attachment(file_date,maintype="application",subtype="pdf",filename=file_name)

    try:
        with smtplib.SMTP("smtp.gmail.com",587) as stmp:
            stmp.ehlo()             
            stmp.starttls()          
            stmp.ehlo()
            stmp.login(sender_email,sender_password)
            stmp.send_message(msg)
            print("Email sent successfully")
    except Exception as e:
        print(f"Failed to send email: {str(e)}")
    
email_tool = FunctionTool(
    name="send_email_with_attachment",
    description="Send an email with an attachment",
    func=send_email_with_attachment
)

NameError: name 'FunctionTool' is not defined

In [22]:
email_sender = AssistantAgent(
    name = "EmailSenderAgent",
    model_client=model_client,
    system_message="""
    You are an email sender agent. Your task is to send an email to the user using the email_sender tool which you have been given.
    You will get the input required to send the email from the previous agent.
    THE FILE IS STORED IN (r"docs\{the_name_of_the_report}.pdf") send like that to tool in the attachment args..
    Say "STOP" after when you are finished.
    """,
    tools = [email_tool],
    reflect_on_tool_use=True
)

In [23]:
task = """ My email id is "mkumari233107@gmail.com" and my name is "USER1222"
The draft for the bill is:-
Data Privacy Regulation Act, 2025

Preamble

An Act to provide for the regulation, protection, and processing of personal data of individuals, and
to establish a Data Protection Authority for effective enforcement and grievance redressal.

Section 1 – Definitions

a) "Data Subject" means any individual whose personal data is being processed.
b) "Data Controller" refers to any individual or organization that determines the purposes and
means of processing personal data.
c) "Processing" includes any operation performed on personal data, such as collection, recording,
storage, adaptation, or alteration.

Section 2 – Rights of Data Subjects

Every data subject shall have the right to:
1. Access their personal data being processed.
2. Request correction of inaccurate or incomplete data.
3. Withdraw consent at any time.
4. Be informed of data breaches affecting their personal information.

Section 3 – Obligations of Data Controllers

1. Ensure that personal data is processed lawfully, fairly, and transparently.
2. Collect data only for specified, explicit, and legitimate purposes.
3. Maintain data accuracy and ensure it is up to date.
4. Implement technical and organizational measures to ensure data security.

Section 4 – Data Protection Authority

A Data Protection Authority (DPA) shall be established to oversee enforcement, conduct
investigations, and issue guidelines.
The DPA shall have powers to impose penalties, summon witnesses, and mandate data audits.

Section 5 – Penalties and Enforcement

Any violation of this Act shall attract monetary penalties up to n10 crores or 2% of annual turnover,
whichever is higher.
In cases of willful misconduct, criminal liability shall be pursued.

Section 6 – Amendments

The Parliament may amend any section of this Act by a simple majority vote with appropriate
justification and review.
Certification

This bill is proposed for review and debate by the legislative assembly. The provisions are subject
to judicial evaluation, political deliberation, and necessary amendment before passage."""

In [ ]:
inner_termination = TextMentionTermination("STOP")

inner_team = RoundRobinGroupChat([bill_parser],termination_condition=inner_termination,max_turns=1)

inner_team2 = SelectorGroupChat(participants=[ruling_pary,oppposing_party,debate_summeriser],
model_client=model_client,
termination_condition=inner_termination,
selector_prompt=custom_selector_prompt,
selector_func=selector_func,
allow_repeated_speaker=False,max_turns=7)

inner_team3 = RoundRobinGroupChat(participants=[judge_agent],termination_condition=inner_termination,max_turns=5)

inner_team4 = RoundRobinGroupChat(participants=[voting_aggregator,amendment_writer,report_maker],termination_condition=inner_termination,max_turns=3)

society_of_mind_agent = SocietyOfMindAgent(name="society_of_mind",team=inner_team,model_client=model_client)

society_of_mind_agent2 = SocietyOfMindAgent(name="society_of_mind2",team=inner_team2,model_client=model_client)

society_of_mind_agent3 = SocietyOfMindAgent(name ="society_of_mind3",team = inner_team3,model_client=model_client)

society_of_mind_agent4 = SocietyOfMindAgent(name="society_of_mind4",team=inner_team4,model_client=model_client)

team = RoundRobinGroupChat([society_of_mind_agent,society_of_mind_agent2,society_of_mind_agent3,society_of_mind_agent4,email_sender],max_turns=8,)

In [25]:
async def main():
    await Console(team.run_stream(task=task))

await main()

---------- TextMessage (user) ----------
 My email id is "mkumari233107@gmail.com" and my name is "USER1222"
The draft for the bill is:-
Data Privacy Regulation Act, 2025

Preamble

An Act to provide for the regulation, protection, and processing of personal data of individuals, and
to establish a Data Protection Authority for effective enforcement and grievance redressal.

Section 1 – Definitions

a) "Data Subject" means any individual whose personal data is being processed.
b) "Data Controller" refers to any individual or organization that determines the purposes and
means of processing personal data.
c) "Processing" includes any operation performed on personal data, such as collection, recording,
storage, adaptation, or alteration.

Section 2 – Rights of Data Subjects

Every data subject shall have the right to:
1. Access their personal data being processed.
2. Request correction of inaccurate or incomplete data.
3. Withdraw consent at any time.
4. Be informed of data breaches affec